## 자전거 대여 수요 예측
#### 2011년 1월부터 2012년 12월까지 날짜/시간, 기온, 습도, 풍속 등의 정보를 기반으로 1시간 간격 동안의 자전거 대여 횟수가 기재되어 있다.

## 다음 코드는 복습과제를 하는데 있어 필요한 모듈입니다. 한 번 꼭 돌려주고 실습해 주세요.

In [28]:
import numpy as np
import pandas as pd
import seaborn as sns

In [29]:
import warnings
warnings.filterwarnings("ignore",category=RuntimeWarning)

## 1. 데이터 불러오기 및 전처리
### (1) bikeshare.csv를 pandas를 이용해 bike_df의 이름으로 저장하고 행렬 개수와 5개의 자료를 뽑아보시오.

In [30]:
bike_df=pd.read_csv("bikeshare.csv")
print(bike_df.shape)
bike_df.head()

(10886, 12)


,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


### (2) bike_df에서 항목별 데이터 타입을 확인하는 코드를 작성하시오.

In [32]:
bike_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    10886 non-null  object 
 1   season      10886 non-null  int64  
 2   holiday     10886 non-null  int64  
 3   workingday  10886 non-null  int64  
 4   weather     10886 non-null  int64  
 5   temp        10886 non-null  float64
 6   atemp       10886 non-null  float64
 7   humidity    10886 non-null  int64  
 8   windspeed   10886 non-null  float64
 9   casual      10886 non-null  int64  
 10  registered  10886 non-null  int64  
 11  count       10886 non-null  int64  
dtypes: float64(3), int64(8), object(1)
memory usage: 1020.7+ KB


### (3) pandas를 이용해 datetime의 칼럼을 년, 월, 일, 시간과 같은 4개의 속성으로 분리하시오.
- 판다스에서 문자열을 datatime 타입으로 변환하는 apply(pd.to_datetime) 메서드를 이용해 datetime이라는 이름의 새로운 칼럼을 만드시오.

In [33]:
bike_df["datetime"]=bike_df.datetime.apply(pd.to_datetime)

- apply() 함수와 lambda를 이용해 새로운 datetime 칼럼에서 년, 월, 일, 시간 칼럼을 새로 만드시오.

In [34]:
bike_df['year']=bike_df.datetime.apply(lambda x:x.year)
bike_df['month']=bike_df.datetime.apply(lambda x:x.month)
bike_df['day']=bike_df.datetime.apply(lambda x:x.day)
bike_df['hour']=bike_df.datetime.apply(lambda x:x.hour)

- 3개의 bike_df를 뽑아 제대로 칼럼들이 생성되었는지 확인하시오.

In [35]:
bike_df.head(3)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,month,day,hour
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16,2011,1,1,0
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40,2011,1,1,1
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32,2011,1,1,2


- datetime 칼럼을 삭제하시오. 또한 casual+registered=count 이므로 casual과 register 칼럼은 따로 필요하지 않으므로 이들도 삭제하시오.

In [36]:
bike_df.drop(['datetime','casual','registered'],axis=1,inplace=True)

In [37]:
bike_df.head()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count,year,month,day,hour
0,1,0,0,1,9.84,14.395,81,0.0,16,2011,1,1,0
1,1,0,0,1,9.02,13.635,80,0.0,40,2011,1,1,1
2,1,0,0,1,9.02,13.635,80,0.0,32,2011,1,1,2
3,1,0,0,1,9.84,14.395,75,0.0,13,2011,1,1,3
4,1,0,0,1,9.84,14.395,75,0.0,1,2011,1,1,4


## 2. 선형 회귀 실습하기

### (1) X_fearues와 y_target이라는 이름으로 독립변수와 종속변수를 분리하시오. 

In [38]:
y_target=bike_df["count"]
X_features=bike_df.drop(["count"],axis=1,inplace=False)

### (2) 숫자형 카테고리 값을 pandas의 get_dummies를 이용해 원-핫 인코딩하시오.
- 현재 데이터타입이 숫자형 카테고리인 칼럼은 year,month,day,hour,holiday,workingday,season,weather 입니다.
- X_features_ohe란 이름에 저장하세요.

In [39]:
X_features_ohe=pd.get_dummies(X_features,columns=['year','month','day','hour','holiday','workingday','season','weather'])

In [40]:
X_features_ohe.head()

,temp,atemp,humidity,windspeed,year_2011,year_2012,month_1,month_2,month_3,month_4,...,workingday_0,workingday_1,season_1,season_2,season_3,season_4,weather_1,weather_2,weather_3,weather_4
0,9.84,14.395,81,0.0,1,0,1,0,0,0,...,1,0,1,0,0,0,1,0,0,0
1,9.02,13.635,80,0.0,1,0,1,0,0,0,...,1,0,1,0,0,0,1,0,0,0
2,9.02,13.635,80,0.0,1,0,1,0,0,0,...,1,0,1,0,0,0,1,0,0,0
3,9.84,14.395,75,0.0,1,0,1,0,0,0,...,1,0,1,0,0,0,1,0,0,0
4,9.84,14.395,75,0.0,1,0,1,0,0,0,...,1,0,1,0,0,0,1,0,0,0


### (3) 사이킷런에서 train_test_split을 이용해 훈련세트와 테스트 세트를 분리하시오.
- 이때 종속변수는 count 칼럼이고 독립변수는 종속변수 이외의 모든 칼럼입니다.
- X_train,X_test,y_train,y_test 이란 이름으로 세트를 만들고 test_size=0.3,random_state=0 이라는 옵션 추가하세요.

In [41]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_features_ohe,y_target,test_size=0.3,random_state=0)

### (4) 회귀 종류별로 실습

#### i. 선형 회귀
- 사이킷런에서 LinearRegression 을 불러와 fit한 후 X_test를 예측하여 pred란 이름에 저장하시오.
- 사이킷런에서의 mean_square_error과 np.sqrt()를 이용해 y_test와 pred의 RMSE를 출력하시오.
- 이 모델의 intercept와 coefficient를 출력하시오.

In [42]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

lr_reg=LinearRegression()
lr_reg.fit(X_train,y_train)
pred=lr_reg.predict(X_test)

np.sqrt(mean_squared_error(y_test,pred))

101.62067160912773

In [43]:
lr_reg.intercept_,lr_reg.coef_

(-965998005422.7465,
 array([ 2.76908441e+00,  1.85782671e+00, -8.03575703e-01, -7.33288778e-01,
         7.67702186e+11,  7.67702186e+11, -3.73632386e+11, -3.73632386e+11,
        -3.73632386e+11, -5.53397678e+11, -5.53397678e+11, -5.53397678e+11,
        -1.30040940e+12, -1.30040940e+12, -1.30040940e+12, -5.28237663e+11,
        -5.28237663e+11, -5.28237663e+11, -1.26446475e+10, -1.26446475e+10,
        -1.26446475e+10, -1.26446475e+10, -1.26446475e+10, -1.26446475e+10,
        -1.26446475e+10, -1.26446475e+10, -1.26446475e+10, -1.26446475e+10,
        -1.26446475e+10, -1.26446475e+10, -1.26446475e+10, -1.26446475e+10,
        -1.26446475e+10, -1.26446475e+10, -1.26446475e+10, -1.26446475e+10,
        -1.26446475e+10, -2.65541047e+10, -2.65541047e+10, -2.65541047e+10,
        -2.65541048e+10, -2.65541048e+10, -2.65541047e+10, -2.65541047e+10,
        -2.65541045e+10, -2.65541044e+10, -2.65541045e+10, -2.65541046e+10,
        -2.65541046e+10, -2.65541045e+10, -2.65541045e+10, -2.65541

#### ii. 확률적 경사 하강법을 이용한 선형 회귀
- 사이킷런에서 제공하는 SGDRegressor을 이용해 위와 같은 과정을 반복하시오.
- 이때 max_iter=1000,tol=1e-3,penalty=None,eta0=0.1 인 옵션을 추가하시오.

In [44]:
from sklearn.linear_model import SGDRegressor

sgd_reg=SGDRegressor(max_iter=1000,tol=1e-3,penalty=None,eta0=0.1)
sgd_reg.fit(X_train,y_train)

np.sqrt(mean_squared_error(y_test,pred))

101.62067160912773

In [45]:
sgd_reg.intercept_,sgd_reg.coef_

(array([-8.00829154e+10]),
 array([-1.42666975e+11, -9.46276980e+10, -1.62559798e+11, -1.94099370e+11,
         1.10886186e+11, -1.90969102e+11, -4.93313925e+10,  2.89587866e+09,
        -2.54320714e+10,  8.47647922e+10,  4.15099143e+10, -1.01515924e+11,
        -6.59605730e+10, -1.08751140e+10,  1.45796991e+11, -1.03770996e+11,
        -6.51835627e+10,  6.70191411e+10,  8.44490814e+10,  1.04160554e+11,
         1.25546079e+11,  1.77736515e+11, -9.70173914e+10, -1.53658778e+11,
         1.07036227e+11, -1.56228994e+11, -2.64212588e+11, -1.32854653e+11,
         6.28173510e+10, -3.49852938e+10,  9.91633969e+10, -1.79184011e+11,
        -9.85098841e+10,  7.66857830e+10, -5.29508281e+10,  1.35675026e+11,
         1.16249493e+11, -4.20341381e+10, -1.83446711e+11, -5.58349903e+10,
         1.05744708e+09,  7.53523695e+10, -6.04785429e+09,  2.95227373e+10,
        -8.15288247e+10, -4.43043090e+10, -1.57657743e+11, -2.99043598e+09,
         9.73706107e+10,  5.79943043e+10, -9.96147381e+10,  4

#### iv. 라쏘와 릿지
- 위와 같은 방식대로 사이킷런에서 Ridge와 Lasso를 불러와 RMSE를 출력하시오.
- 각각의 intercept, coefficient를 출력하시오.
- Ridge와 Lasso에서 alpha=0.1 이라는 옵션을 추가하시오.

In [46]:
from sklearn.linear_model import Ridge, Lasso

ridge_reg=Ridge(alpha=0.1)
ridge_reg.fit(X_train,y_train)
pred=ridge_reg.predict(X_test)

np.sqrt(mean_squared_error(y_test,pred))

101.61963796032045

In [47]:
ridge_reg.intercept_,ridge_reg.coef_

(93.59951479931932,
 array([ 2.77635084e+00,  1.85638810e+00, -8.04848466e-01, -7.32755619e-01,
        -4.30350512e+01,  4.30350512e+01, -2.78809868e+01, -1.12490848e+01,
         6.11830405e+00, -9.97383401e+00,  1.52190326e+01,  2.91904860e+00,
        -1.08898027e+01, -1.00814072e+01,  2.46162001e+01,  2.21827929e+01,
         1.88035315e-01, -1.16829806e+00, -9.80464362e+00, -2.81919690e+00,
         1.30083731e+00,  8.68240255e-01,  2.18864828e+00,  4.24340314e-01,
        -7.62089700e+00, -9.53377117e+00,  7.17480447e-02,  1.02674291e+00,
         2.07627610e+00, -5.06884674e+00,  2.13600282e+00,  2.81138083e+00,
         6.74384093e+00,  2.20615475e+00,  1.81487720e+01, -6.33769375e+00,
         1.18206464e+00, -1.24689439e+02, -1.45447914e+02, -1.59260535e+02,
        -1.66295508e+02, -1.68110470e+02, -1.49564140e+02, -9.42788975e+01,
         4.05901985e+01,  1.92968022e+02,  4.08782973e+01, -1.66084925e+01,
         5.55252726e+00,  5.22344891e+01,  4.69859712e+01,  3.274547

In [48]:
lasso_reg=Lasso(alpha=0.1)
lasso_reg.fit(X_train,y_train)
pred=lasso_reg.predict(X_test)

np.sqrt(mean_squared_error(y_test,pred))

101.56225421214636

In [49]:
lasso_reg.intercept_,lasso_reg.coef_

(218.6524011948083,
 array([ 3.27180766e+00,  1.76978059e+00, -8.73804775e-01, -7.10780296e-01,
        -8.50381823e+01,  0.00000000e+00, -2.96980105e+01, -1.35728022e+01,
         0.00000000e+00, -4.87987262e+00,  1.71401016e+01,  2.85040566e+00,
        -5.27072118e+00, -3.75178267e+00,  3.03745171e+01,  1.90334989e+01,
         0.00000000e+00, -0.00000000e+00, -8.66790918e+00, -1.84686122e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        -6.39095552e+00, -8.72523168e+00, -0.00000000e+00,  0.00000000e+00,
         0.00000000e+00, -3.85360130e+00,  0.00000000e+00,  1.87183492e-01,
         3.68292429e+00,  0.00000000e+00,  1.48958750e+01, -4.98174542e+00,
         0.00000000e+00, -1.49473706e+02, -1.70097066e+02, -1.83473824e+02,
        -1.90307400e+02, -1.91967177e+02, -1.73459149e+02, -1.18167091e+02,
         1.18118347e+01,  1.63673667e+02,  1.08736427e+01, -4.25399488e+01,
        -2.08513802e+01,  2.03154895e+01,  1.47011803e+01,  1.274470